In [1]:
import re
import pandas as pd
import numpy as np
import PyPDF2
import os

define a function to get pdf as txt

In [2]:
def getPdfContent(filename):
    pdfFile = open(filename,'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFile)
    content = ""
    for i in range(0, pdfReader.numPages):
        page = pdfReader.getPage(i)

        extractedText = page.extractText()
        content += extractedText + "\n"
        # return content.encode("ascii", "ignore")
    return content

use a file path to see all the pdfs

In [118]:
DATA_DIR="/Users/yaohuang/Downloads/receipt"
dir_files = os.listdir(DATA_DIR)


In [119]:
full_count = pd.DataFrame(columns=[])

In [120]:

for f in dir_files: 
    if os.path.splitext(f)[1]=='.pdf':
        receipt=getPdfContent(os.path.join(DATA_DIR,f))
        pattern = re.compile(r'19\d{10}')   # 查找数字
        result = pattern.findall(receipt)
        print(len(result))
        receipt_series = pd.Series(result)
        frame = { 'upc': receipt_series} 
        resultpd = pd.DataFrame(frame)
        data1 = resultpd.groupby(by='upc')['upc'].count()
        pattern2 = re.compile(r'\€[a-zA-Z]+\€')
        receipt_count = pd.DataFrame({ pattern2.findall(receipt)[0]: data1})
        full_count=pd.concat([receipt_count,full_count],axis=0)
full_count = full_count.groupby(by='upc').sum()
full_count['row_sum'] = full_count.apply(lambda x: x.sum(), axis=1)
full_count=full_count.reset_index()

16
3
22
8
11
25
3
23


of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [ipykernel_launcher.py:13]


25
18
37
29
18
18


In [121]:
full_count

,upc,€Concord€,€Gaffney€,€NORFOLK€,€Pittsburgh€,€Savannah€,€Sevierville€,€Waipahu€,row_sum
0,192317777413,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,192317871784,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0
2,192877121305,6.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0
3,192877322085,0.0,0.0,6.0,0.0,0.0,0.0,0.0,6.0
4,192877329084,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,192877334996,0.0,0.0,0.0,35.0,72.0,23.0,36.0,166.0
6,192877642145,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
7,192877672173,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0
8,192877681700,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
9,192877682004,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


set up a upc dictionary which could be used every time

In [102]:
# initialize list of lists 
data = [["192317777413", "kimberly水桶黑拼"], ["192877121305", "三合一 黑色"]] 
  
# Create the pandas DataFrame 
upc_translate = pd.DataFrame(data, columns = ['upc', 'name']) 
  
# print dataframe. 
upc_translate

,upc,name
0,192317777413,kimberly水桶黑拼
1,192877121305,三合一 黑色


combine the two dataframe on left

In [122]:
pd.merge(full_count,upc_translate,how="left",on="upc")

,upc,€Concord€,€Gaffney€,€NORFOLK€,€Pittsburgh€,€Savannah€,€Sevierville€,€Waipahu€,row_sum,name
0,192317777413,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,kimberly水桶黑拼
1,192317871784,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,NaN
2,192877121305,6.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0,三合一 黑色
3,192877322085,0.0,0.0,6.0,0.0,0.0,0.0,0.0,6.0,NaN
4,192877329084,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
5,192877334996,0.0,0.0,0.0,35.0,72.0,23.0,36.0,166.0,NaN
6,192877642145,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
7,192877672173,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,NaN
8,192877681700,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN
9,192877682004,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
